In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 

In [2]:
train = pd.read_csv('../apartment/train.csv')
#train.info()

In [3]:
from sklearn import preprocessing
catagorical_cols=[]
for column in train.columns.values:
    if (train[column].dtype=='O'):
        catagorical_cols.append(column)
for x in catagorical_cols:
    train[x]=train[x].fillna('None')
for x in catagorical_cols:
    label = preprocessing.LabelEncoder()
    train[x] = label.fit_transform(train[x])
#train.info()
for x in train.columns.values:
    train[x].values.astype(np.float64)
#train.info()

In [4]:
#train[train.columns.values].isnull().sum()
for x in train.columns.values:
    if(train[x].isnull().sum() >0):
        print(x)

LotFrontage
MasVnrArea
GarageYrBlt


In [5]:
#method1

In [5]:
#train['LotFrontage'].describe()
#259

count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [6]:
#train['MasVnrArea'].describe()
#8

count    1452.000000
mean      103.685262
std       181.066207
min         0.000000
25%         0.000000
50%         0.000000
75%       166.000000
max      1600.000000
Name: MasVnrArea, dtype: float64

In [7]:
#train['GarageYrBlt'].describe()
#81

count    1379.000000
mean     1978.506164
std        24.689725
min      1900.000000
25%      1961.000000
50%      1980.000000
75%      2002.000000
max      2010.000000
Name: GarageYrBlt, dtype: float64

In [8]:
#train['LotFrontage'].fillna(70.049958,inplace = True)
#train['MasVnrArea'].fillna(103.685262,inplace = True)
#train['GarageYrBlt'].fillna(1978.506164,inplace = True)

In [5]:
# method2

In [5]:
train = train.drop('LotFrontage',1)
train = train.drop('MasVnrArea',1)
train = train.drop('GarageYrBlt',1)

In [6]:
#train[train.columns.values].isnull().sum()

In [6]:
train1 = train.drop('SalePrice',1)
#train1.info()


In [7]:
from sklearn import preprocessing
from sklearn import linear_model, svm, gaussian_process
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso
import numpy as np

In [8]:
cols = train1.columns.values
X = train1[cols].values.astype(np.float64)
y = train['SalePrice'].values.astype(np.float64)
#X_scaled = preprocessing.StandardScaler().fit_transform(x)
#y_scaled = preprocessing.StandardScaler().fit_transform(y.reshape(-1,1))
#minmaxscaler = preprocessing.MinMaxScaler(copy=True, feature_range=(0, 1)).fit(X)
#X_scaled = minmaxscaler.fit_transform(X)
#y_scaled = minmaxscaler.fit_transform(y.reshape(-1,1))

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

minmaxscaler = preprocessing.MinMaxScaler(copy=True, feature_range=(0, 1)).fit(X)
X_train = minmaxscaler.fit_transform(X_train)
#y_train = minmaxscaler.fit_transform(y_train.reshape(-1,1))
X_test = minmaxscaler.transform(X_test)
#y_test = minmaxscaler.transform(y_test.reshape(-1,1))

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)


In [10]:
X_train = pca.fit_transform(X_train)
print(pca.explained_variance_ratio_)
#y_train = pca.fit_transform(y_train.reshape(-1,1))

[ 0.15975135  0.07439591  0.06130991  0.04650609  0.04086365  0.03883997
  0.03407173  0.0329152 ]


In [11]:
X_test = pca.transform(X_test)
#y_test = pca.transform(y_test.reshape(-1,1))

In [12]:
clfs = {
        'svm':svm.SVR(), 
        'RandomForestRegressor':RandomForestRegressor(n_estimators=400),
        'LinearRegression':linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
        'DecisionTreeRegressor':DecisionTreeRegressor(),
        'SGDRegressor':SGDRegressor(max_iter=5, tol = None),
        'GradientBoostingRegressor':GradientBoostingRegressor(),
        'MLPRegressor':MLPRegressor(solver='adam', activation = 'relu',hidden_layer_sizes=(70,65,60,55,50,45,40,35,30,25,20,15,10,5), max_iter=1000, learning_rate='adaptive', random_state= None),
        'Lasso':Lasso(alpha = 0.1,max_iter=1000,random_state=None)
       }
for clf in clfs:
    try:
        clfs[clf].fit(X_train, y_train.ravel())   #why need to make this ravel()??
        y_pred = clfs[clf].predict(X_test)
        #print(clf + " cost:" + str(np.sum(y_pred-y_test)/len(y_pred)))
        #print(clf + " score:" + str(np.mean(np.absolute(y_pred-y_test))))
        #print(clf + " score:" + str(np.mean((y_pred-y_test)*(y_pred-y_test))))
        print(clf + " score:" + str(np.sqrt(np.mean((np.log(y_pred)-np.log(y_test))*(np.log(y_pred)-np.log(y_test))))))
    except Exception as e:
        print(clf + " Error:")
        print(str(e))

svm score:0.411565711579
RandomForestRegressor score:0.214377791259
LinearRegression score:0.23172619872
DecisionTreeRegressor score:0.305350887557
SGDRegressor score:0.233431409543
GradientBoostingRegressor score:0.224454542134
MLPRegressor score:0.220416165551
Lasso score:0.231726368894


In [ ]:
#pca = PCA(n_components=8)
#train1 = pca.fit(train)
#data_reduce = pca.fit_transform(train)
#data_reduce
